In [1]:
pip install sentence-transformers


  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.34.0 from https://files.pythonhosted.org/packages/d9/b7/98f821d70102e2d38483bbb7013a689d2d646daa4495377bc910374ad727/transformers-4.41.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------------------- 43.8/43.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub>=0.15.1 from https://files.pythonhosted.org/packages/69/d6/73f9d1b7c4da5f0544bc17680d0fa9932445423b90cd38e1ee77d001a4f5/huggingface_hub-0.23.4-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/8f/df/de2c06b316142063b6ccccc97cdc54185e3af771aa4f056d56f0db0e3466/fss

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2024.6.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [3]:
# Load your data
data = pd.read_excel("email_sample.xlsx")

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.body, data.label, test_size=0.25, random_state=42)


In [4]:
# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Minfy.DESKTOP-QM40AOM\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Minfy.DESKTOP-QM40AOM\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

c:\Users\Minfy.DESKTOP-QM40AOM\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
# Function to encode the text using BERT
def encode_texts(texts, tokenizer, model, max_length=128):
    # Tokenize and encode the texts
    inputs = tokenizer(texts.tolist(), return_tensors='pt', max_length=max_length, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (we'll use the [CLS] token embedding as a summary)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

In [6]:

# Encode the training and test data
X_train_encoded = encode_texts(X_train, tokenizer, model)
X_test_encoded = encode_texts(X_test, tokenizer, model)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 295698432 bytes.

# ---- Naive Bayes Classifier ----

In [ ]:
# Scale the embeddings to be non-negative for MultinomialNB
scaler_nb = MinMaxScaler()
X_train_encoded_nb = scaler_nb.fit_transform(X_train_encoded)
X_test_encoded_nb = scaler_nb.transform(X_test_encoded)

In [ ]:
# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_encoded_nb, y_train)

# Predict on the test set using Naive Bayes
y_pred_nb = nb_classifier.predict(X_test_encoded_nb)

In [ ]:
# Calculate accuracy, precision, and recall for Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='weighted')
recall_nb = recall_score(y_test, y_pred_nb, average='weighted')

print(f"Naive Bayes - Accuracy: {accuracy_nb}")
print(f"Naive Bayes - Precision: {precision_nb}")
print(f"Naive Bayes - Recall: {recall_nb}")

Naive Bayes - Accuracy: 0.8968253968253969
Naive Bayes - Precision: 0.8970562843570685
Naive Bayes - Recall: 0.8968253968253969


# ---- Random Forest Classifier ----

In [ ]:
# Standardize the embeddings for RandomForest
scaler_rf = StandardScaler()
X_train_encoded_rf = scaler_rf.fit_transform(X_train_encoded)
X_test_encoded_rf = scaler_rf.transform(X_test_encoded)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_encoded_rf, y_train)

# Predict on the test set using Random Forest
y_pred_rf = rf_classifier.predict(X_test_encoded_rf)

In [ ]:
# Calculate accuracy, precision, and recall for Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest - Accuracy: {accuracy_rf}")
print(f"Random Forest - Precision: {precision_rf}")
print(f"Random Forest - Recall: {recall_rf}")

Random Forest - Accuracy: 0.8888888888888888
Random Forest - Precision: 0.8899176954732511
Random Forest - Recall: 0.8888888888888888


# ---- XgBoost Classifier ----

In [ ]:
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_classifier.fit(X_train_encoded_rf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Predict on the test set using XGBoost
y_pred_xgb = xgb_classifier.predict(X_test_encoded_rf)

# Calculate accuracy, precision, and recall for XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')

In [ ]:
print(f"XGBoost - Accuracy: {accuracy_xgb}")
print(f"XGBoost - Precision: {precision_xgb}")
print(f"XGBoost - Recall: {recall_xgb}")

XGBoost - Accuracy: 0.9206349206349206
XGBoost - Precision: 0.9239514239514239
XGBoost - Recall: 0.9206349206349206
